# 0. Introduction

## Goal

This notebook aims to present how data stored in non formatted documents could be leveraged to improve data quality inside the PIM.

This notebook uses a handful of modules developped inside this project.

## What pipeline?

The different steps for this project are as follows:

1. fetch all product IDs from PIM with the associated ingredient lists
- split the products between a train set and a test set
- train the algorithm on the train set: i.e. construct the vocabulary
- make it make prediction on the test set
- compare it with the ingredient list on this product

# 1. Fetching the data

We will use production data for training and testing of this model. The ID of the products are the PIM uid, and therefore are listed in the directory of the PIM-API module.

First, let's get those uids.

In [ ]:
from src import pimapi
requester = pimapi.Requester('prd')
requester.refresh_directory()
requester._directory

One can see the modification status of the product via the `modification_report` method:

In [ ]:
requester.modification_report()

Outdated products can be refreshed via the following:

In [ ]:
requester.fetch_list_from_PIM(requester.modified_items(), batch_size=20)
requester.dump_data_from_result()
requester.dump_files_from_result()
requester.modification_report()

The PIM uids of the products are the keys of the `directory` of our requester. We extract the ingredients associated with these uids.

In [ ]:
requester.fetch_all_from_PIM(page_size=1000, max_page=-1, nx_properties='*')
requester.result[0].json()['entries'][0]

In [ ]:
mapping = {'uid': 'uid', 'Libellé': 'title', 'Ingrédients': 'properties.pprodc:ingredientsList'}
df = requester.result_to_dataframe(record_path='entries', mapping=mapping, index='uid')
df

# 2. Train / Test split

We will separate our data into a train test and a test set of equal sizes.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.5, random_state=42)

In [ ]:
print(len(X_train))
X_train.iloc[:10]

In [ ]:
print(len(X_test))
X_test.iloc[:10]

# 3. Constructing the vocabulary

We will now use bag-of-words related functionalities of scikit-learn to construct our vocabulary.

## 3.1 Removing `None` values

First step is to remove `None` values from ingredient lists to make our count of words.

In [ ]:
import pandas as pd
print(f'None values before replacement in X_train: {sum(pd.isna(X_train["Ingrédients"]))}')
X_train.loc[:, 'Ingrédients'].fillna('', inplace=True)
print(f'None values after replacement in X_train: {sum(pd.isna(X_train["Ingrédients"]))}')
print(f'None values before replacement in X_test: {sum(pd.isna(X_test["Ingrédients"]))}')
X_test.loc[:, 'Ingrédients'].fillna('', inplace=True)
print(f'None values after replacement in X_train: {sum(pd.isna(X_test["Ingrédients"]))}')

## 3.2 Parsing the corpus

We now parse our ingredient lists, with a naive approach (no stop words, no preprocessing, ...).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.loc[:, 'Ingrédients'])

We can see that we know have a matrix with as much rows as the number of products in our train corpus, and as much columns as the number of different words in their ingredient lists.

In [ ]:
X_train_counts.shape

The vocabulary has been computed: 

In [ ]:
print(f'The vocabulary length is {len(count_vect.vocabulary_)}')
count_vect.vocabulary_

We can print out the top 10 most frequent words in our ingredient lists:

In [ ]:
word_counts = X_train_counts.sum(axis=0)
word_counts2 = [(word, word_counts[0, idx]) for word, idx in count_vect.vocabulary_.items()]
word_counts2.sort(key=lambda x: x[1], reverse=True)
word_counts = word_counts2
word_counts[:10]

# 4. First analysis of a single document

## 4.1 Parsing a doc from the test set

First, we use a function that parses a document from the disk from its product uid and returns a list of strings. For illustration, we choose one of the products in our test set.

In [ ]:
X_test.iloc[:10]

In [ ]:
from src.pimpdf import PDFDecoder
import os

# This uid has been gotten from the previous cell, maybe from a previous run!
uid = '776613db-a461-44e1-ab6a-1344ac6ae99c'
test_doc_blocks = PDFDecoder.path_to_blocks(os.path.join('.', 'dumps', 'prd', uid, 'FTF.pdf'))
print(f'Number of blocks in this document: {len(test_doc_blocks)}')
test_doc_blocks

For this specific document (*776613db-a461-44e1-ab6a-1344ac6ae99c*), the correct block of text is:

    Ingrédients:  sirop de glucose; sucre; gélatine; dextrose; acidifiants: acide citrique, acide malique; agent d'enrobage: cire de carnauba; correcteurs d'acidité: citrate tricalcique, malate acide de sodium; arôme; concentrés de fruits et de plantes: citron, carthame, spiruline, patate douce, radis; sirop de sucre inverti; colorants: carmins, bleu patenté V, carotènes végétaux, lutéine, anthocyanes.

The index of this correct block is *9* in our block list.

In [ ]:
true_idx = 9

We will now parse these blocks with the vocabulary computed from our train set. We reuse the `CountVectorizer` we trained before, but take care just to use the `transform` method.

Using the `fit_transform` method would retrain the model with the current blocks of text.

In [ ]:
test_doc_counts = count_vect.transform(test_doc_blocks)
test_doc_counts

## 4.2 Getting some insights from this first analysis

We can compute and draw the terms counts for our blocks: 

In [ ]:
import numpy as np

term_counts = np.ravel(test_doc_counts.sum(axis=1))
term_counts

We can see that the *true* ingredient list has the higher term_count. However, term counts alone are likely to have a biais toward long blocks, so we can also compute a term frequency.

We will instantiate a new count_vectorizer, for the sole purpose of counting tokens in the blocks.

In [ ]:
blocks_word_counts = np.ravel(CountVectorizer().fit_transform(test_doc_blocks).todense().sum(axis=1))
blocks_word_counts

We can now compute the frequencies of "ingredient words" in the blocks of this document.

In [ ]:
term_freqs = np.divide(term_counts, blocks_word_counts, out=np.zeros(term_counts.shape), where=blocks_word_counts!=0)
term_freqs

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

colors = ['other'] * len(test_doc_blocks)
colors[true_idx] = 'ingredients'

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)
sns.barplot(ax=ax[0], x=term_counts, y=[block[:10] for block in test_doc_blocks], hue=colors, dodge=False)
sns.barplot(ax=ax[1], x=term_freqs, y=[block[:10] for block in test_doc_blocks], hue=colors, dodge=False)
ax[0].set_title('Terms counts', fontsize=20)
ax[0].set_ylabel('Blocks', fontsize=18)
ax[1].set_title('Terms frequencies', fontsize=20)
ax[1].xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
pass

We can see that some very short texts also have an "ingredient word frequency" equal to 100%.

We can draw a scatter plot of these indicators:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
sns.scatterplot(ax=ax, x=term_counts, y=term_freqs, hue=colors)
ax.set_title('Ingredient word frequency vs. count', fontsize=20)
ax.set_xlabel('Ingredient word count in block', fontsize=16)
ax.set_ylabel('Ingtredient word frequency in block', fontsize=16)
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
pass

As expected, the correct ingredient list is in the top right quadrant of this representation. Should all cases yield results as sharply contrasted, we will have no difficulty in constructing an accurate functionality!

## 4.3 A difficulty arises in assessing ground truth for model validation

### Long strings are seldom strictly equal

We can compare the document ingredient list, with the one stored in the PIM system:

In [ ]:
test_doc_blocks[true_idx].replace('\n', '') == df.loc[uid, "Ingrédients"]

Because a great deal of the current data in the PIM system has (at least once!) been manually keyed in, it is very likely that there will be an arguably high ratio of mistakes.

In [ ]:
print('From pdf file: ')
print(test_doc_blocks[true_idx].replace('\n', ''))
print('--------------------------------------------------------------------------------------')
print('From PIM system:')
print(df.loc[uid, "Ingrédients"])

One can see that although very close, those 2 texts are somewhat different, from the punctuation marks.

### Different strategies can be undertaken to get around this difficulty

It is mandatory to compare the results of the model with the ground truth to assess the performance of the model. Some workarounds can be set up:

- Ignoring all products that do not have a strict matching block in their pdf file:
    - This will enable for a simple validation process
    - But it might dramatically decreasing the size of our dataset
    - as well as making 'short ingredient list' product overrepresented
    
    
- Defining a softer way to match texts between pdf files and PIM system data **with some text preprocessing** and filtering products that do not have a matching block
    - This will mitigate the previous drawbacks
    - But will increase complexity
    
    
- Defining a softer way to match texts between pdf files and PIM system data **by computing an edition distance** and filtering products that do not have a matching block
    - This will mitigate the previous drawbacks
    - But will increase complexity, as well as requiring to manually set up a distance threshold.
    - This could also lead to have separate pdf file blocks considered ground truth should the threshold distance be too high
    
- Considering blocks n-grams 
    - This might increase the number of 

- Manually labeling some pdf files
    - The most efficient
    - But the most time-consuming too!


# 5. Comparison between PIM system *ground truth* with documents content

We can try to find the products for which the PIM system ingredient list is strictly equal to one of the pdf file blocks.

## 5.1 Retrieving all the blocks from our corpus

The function below enables to retrieve all the blocks as a pandas Series.

In [ ]:
%%time
uid_list = list(requester._directory.index)
path_list = [os.path.join('.', 'dumps', 'prd', uid, 'FTF.pdf') for uid in uid_list]
path_series = pd.Series(path_list, index=uid_list)
blocks_series = PDFDecoder.threaded_paths_to_blocks(path_series)

In [ ]:
blocks_series.rename('pdf_blocks', inplace=True)
blocks_series.index.rename('uid', inplace=True)
blocks_series

As it takes some time to run the pdf parsing on all the corpus, we save it in a csv.

In [ ]:
import datetime
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
blocks_series.to_csv(os.path.join('.', 'dumps', 'prd', 'blocks_'+ timestamp + '.csv'), header=True)

## 5.2 Comparing blocks with PIM ingredient lists

We only keep products with ingredient list.

In [ ]:
df_ingred = df.loc[pd.notna(df['Ingrédients'])]
df_ingred

Among these products with an ingredient list, we only keep the ones which have (at least) a block from the pdf that is strictly matching its PIM ingredient list:

In [ ]:
joined_df = df_ingred.join(blocks_series)
matching = joined_df.loc[joined_df['Ingrédients'] == joined_df['pdf_blocks']].drop_duplicates()
matching

We can see that only keeping products with strict equality between PIM ingredient list and any of the pdf blocks has dramatically reduced their number: from 9549 to 299. New sample size only represents less than 3.5% of the initial set.

## 5.3 Making the same comparison but softening the criterion

TODO !!!

# 6. Manually labeling data

The most straightforward solution to have labeled data (but also the most time consuming...) is to label them manually!

## 6.1 Randomly selecting products to label

First step is to **randomly** select products from PIM, extract their attached documents and store them away safely. The criterion for these 500 products will be:

- they are food products: beverages or grocery
- they have a supplier technical datasheet attached

They will be stratified by product type (beverage or grocery).

In [ ]:
requester.result[0].json()['entries'][0]

In [ ]:
mapping = {'uid': 'uid',
           'designation': 'title',
           'state': 'state',
           'ingredients': 'properties.pprodc:ingredientsList',
           'type': 'properties.pprodtop:typeOfProduct'}
df = requester.file_report_from_result(mapping=mapping, index='uid') # , record_path='entries') 
df

In [ ]:
filtered_df = df.loc[(df.type.isin(['grocery', 'nonAlcoholicDrink']))
                     & (df.has_supplierdatasheet)]

In [ ]:
train, ground_truth_df = train_test_split(filtered_df, test_size=500, random_state=42, stratify=filtered_df.type)
ground_truth_df

Now that we have our products, we save their attached documents on disk.

In [ ]:
requester.fetch_list_from_PIM(ground_truth_df.index, batch_size=20)

In [ ]:
requester.dump_data_from_result(update_directory=False, root_path=os.path.join('.', 'ground_truth'))
requester.dump_files_from_result(update_directory=False, root_path=os.path.join('.', 'ground_truth'))

In [ ]:
ground_truth_df['designation'].to_csv(os.path.join('.', 'ground_truth', 'uid.csv'), header=True, encoding='utf-8-sig')

We then try to reimport the ground truth after having processed some files, just to check that everything went ok.

In [ ]:
import pandas as pd
import os

In [ ]:
pd.read_csv(os.path.join('.', 'ground_truth', 'manually_labelled_ground_truth.csv'),
            sep=';',
            encoding='latin-1',
            index_col='uid')

In [ ]:
uid = '70500268-802d-4211-93ba-9edbf6e0e7a3'
print(pd.read_csv(os.path.join('.', 'ground_truth', 'manually_labelled_ground_truth.csv'), sep=';', encoding='latin-1').set_index('uid', drop=True).loc[uid, 'ingredients'])

## 6.2 Reconstructing Train/Test split after labelling

We will now reimport the ground truth csv file and recreate the train/test split.

In [ ]:
test_set =pd.read_csv(os.path.join('.', 'ground_truth', 'manually_labelled_ground_truth.csv'),
                      sep=';',
                      encoding='latin-1',
                      index_col='uid')
test_set

In [ ]:
mapping = {'uid': 'uid',
           'Libellé': 'title',
           'Ingrédients': 'properties.pprodc:ingredientsList'}
df = requester.result_to_dataframe(record_path='entries', mapping=mapping, index='uid')
df

In [ ]:
X_train = df[(~df.index.isin(test_set.index)) & (pd.notna(df['Ingrédients']))]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.loc[:, 'Ingrédients'])
count_vect.vocabulary_

## Different strategies

TODO !!!

But The similarity between the ground truth (the pdf file) and the content of the PIM system can be measured via the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance). This distance is the number of character insertions, deletions or substitutions to get from one text to the other.

If we compute this distance between the pdf file block and the PIM system content we get:

In [ ]:
import jellyfish
dist = jellyfish.levenshtein_distance(test_doc_blocks[true_idx].replace('\n', ''),
                                      df.loc[uid, "Ingrédients"])
print(f'Levenshtein distance between pdf file and PIM system content is: {dist}')

We can compute this distance for each block in our pdf file, and plot it in a bar graph:

In [ ]:
distances = list(map(lambda x:jellyfish.levenshtein_distance(x.replace('\n', ''), df.loc[uid, "Ingrédients"]),
                 test_doc_blocks))
fig, ax = plt.subplots(figsize=(12, 10))
sns.barplot(ax=ax, x=distances, y=[block[:10] for block in test_doc_blocks], hue=colors, dodge=False)
ax.set_title('Levenshtein distance by block', fontsize=20)
ax.set_ylabel('Blocks', fontsize=18)
pass

## 4.2 Making prediction on a document